# MATH2603 Lab — Shortest Paths in Weighted Graphs (Dijkstra)

**Theme (aligns with lecture):** weighted graphs, shortest-path distance, Dijkstra’s algorithm, priority queue.

**What you will produce today:**  
- a working implementation of Dijkstra (with path reconstruction)  
- a few plots / tables  
- short written answers in the **Markdown “Your answer here”** boxes

> **Important:** Dijkstra assumes **all edge weights are non‑negative**. We will test what breaks if this is not true.


## 0. Setup check (5–10 min)

Run the cell below. If you see `ModuleNotFoundError`, install the missing package (ask the tutor if needed).

In [ ]:
import sys, platform
print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

print("numpy:", np.__version__)
print("matplotlib:", plt.matplotlib.__version__)
print("networkx:", nx.__version__)


## 1. A small weighted graph (10–15 min)

We’ll start with a small example so we can *see* what Dijkstra is doing.

We represent a weighted graph as:
- an edge list `[(u,v,w), ...]` where `w` is the weight
- and also as a NetworkX graph for plotting/verification.

In [ ]:
# A small weighted, undirected graph example
edges = [
    ("A","B",4),
    ("A","C",2),
    ("B","C",1),
    ("B","D",5),
    ("C","D",8),
    ("C","E",10),
    ("D","E",2),
    ("D","F",6),
    ("E","F",3),
]

G = nx.Graph()
for u,v,w in edges:
    G.add_edge(u,v,weight=w)

pos = nx.spring_layout(G, seed=1)
plt.figure(figsize=(6,4))
nx.draw(G, pos, with_labels=True, node_size=800)
labels = nx.get_edge_attributes(G, "weight")
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
plt.title("Weighted graph example")
plt.axis("off")
plt.show()

print("Nodes:", list(G.nodes()))
print("Edges:", list(G.edges(data=True))[:3], "...")


### Task 1A (short answer)

1. If we ignore weights, what is the *fewest-edge* path from A to F?  
2. When we include weights, do you expect the optimal path to change? Why?

**Your answer here:**  
- 


## 2. Dijkstra idea: “best known distance so far” (10–15 min)

Dijkstra maintains:
- `dist[v]`: best known distance from `source` to `v`
- `prev[v]`: predecessor pointer for reconstructing the route
- a **priority queue** of `(distance, vertex)` so we always expand the currently-closest unexplored vertex.

We’ll implement a *teaching version* first (clear > fast).

In [ ]:
import heapq

def dijkstra(G, source):
    # initialise
    dist = {v: float("inf") for v in G.nodes()}
    prev = {v: None for v in G.nodes()}
    dist[source] = 0.0

    # priority queue holds (distance, vertex)
    pq = [(0.0, source)]
    visited = set()

    while pq:
        d, u = heapq.heappop(pq)
        if u in visited:
            continue
        visited.add(u)

        # relax edges out of u
        for v, attrs in G[u].items():
            w = attrs.get("weight", 1.0)
            if w < 0:
                raise ValueError("Dijkstra requires non-negative weights.")
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                prev[v] = u
                heapq.heappush(pq, (dist[v], v))
    return dist, prev

def reconstruct_path(prev, target):
    path = []
    cur = target
    while cur is not None:
        path.append(cur)
        cur = prev[cur]
    path.reverse()
    return path


### Task 2A (code + check)

Run Dijkstra from `A` and reconstruct the best path to `F`.

In [ ]:
dist, prev = dijkstra(G, "A")
path_AF = reconstruct_path(prev, "F")
dist_AF = dist["F"]

print("Shortest distance A -> F =", dist_AF)
print("Path:", path_AF)

# Compare with NetworkX built-in (for verification)
nx_path = nx.shortest_path(G, "A", "F", weight="weight")
nx_len  = nx.shortest_path_length(G, "A", "F", weight="weight")
print("NetworkX path:", nx_path)
print("NetworkX length:", nx_len)


### Task 2B (short answer)

Why does the priority queue matter? Write 3–5 sentences explaining what could go wrong if we always pick a random next vertex to expand.

**Your answer here:**  
- 


## 3. Step-by-step tracing (priority queue) (20–30 min)

The PPT shows iterations where we:
1. **dequeue** the smallest-distance vertex
2. **relax** its outgoing edges (try improving neighbours)
3. push updates into the priority queue

Let’s make a trace function that prints the queue and `dist` changes so you can match the PPT logic.

In [ ]:
def dijkstra_trace(G, source, max_steps=20):
    import heapq
    dist = {v: float("inf") for v in G.nodes()}
    prev = {v: None for v in G.nodes()}
    dist[source] = 0.0

    pq = [(0.0, source)]
    visited = set()
    step = 0

    def pq_view(pq):
        # return a sorted view (without destroying pq)
        return sorted([(round(d,3), v) for d,v in pq])

    while pq and step < max_steps:
        step += 1
        d, u = heapq.heappop(pq)
        if u in visited:
            continue
        visited.add(u)

        print(f"--- Iteration {step}: pop ({u}, dist={d}) ---")
        print("PQ before relax:", pq_view(pq))

        for v, attrs in G[u].items():
            w = attrs.get("weight", 1.0)
            if dist[u] + w < dist[v]:
                old = dist[v]
                dist[v] = dist[u] + w
                prev[v] = u
                heapq.heappush(pq, (dist[v], v))
                print(f"  relax {u}->{v} (w={w}): dist[{v}] {old} -> {dist[v]}")

        print("PQ after relax: ", pq_view(pq))
        print("Visited:", visited)
        print()
    return dist, prev

# Try tracing from A (you can increase max_steps if you want)
_ = dijkstra_trace(G, "A", max_steps=10)


### Task 3A (short answer)

Match the trace output to the PPT idea of “Iteration 1, 2, …”.  
- What is the **first** vertex popped after the source?  
- Give one example of an edge relaxation that improves a distance.

**Your answer here:**  
- 


## 4. Implement Dijkstra on your own graph (25–35 min)

Now you will:
1. Create a new weighted graph (6–10 nodes) **of your choice**
2. Run your `dijkstra` function
3. Visualise the best path between two chosen nodes

In [ ]:
# Task 4A: build your own graph here
# Tip: keep weights positive (>= 1)

my_edges = [
    # ("X","Y",3),
]

H = nx.Graph()
for u,v,w in my_edges:
    H.add_edge(u,v,weight=w)

print("Number of nodes:", H.number_of_nodes())
print("Number of edges:", H.number_of_edges())


In [ ]:
# Task 4B: choose source & target
# (change these after you add edges)
source = None
target = None

if source is not None and target is not None:
    distH, prevH = dijkstra(H, source)
    path = reconstruct_path(prevH, target)
    print("Distance:", distH[target])
    print("Path:", path)
else:
    print("Set source and target after defining my_edges.")


In [ ]:
# Task 4C: plot your graph and highlight the shortest path
def plot_graph_with_path(G, path=None, title=""):
    pos = nx.spring_layout(G, seed=2)
    plt.figure(figsize=(6,4))
    nx.draw(G, pos, with_labels=True, node_size=800)
    labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

    if path is not None and len(path) >= 2:
        path_edges = list(zip(path[:-1], path[1:]))
        nx.draw_networkx_edges(G, pos, edgelist=path_edges, width=4)
    plt.title(title)
    plt.axis("off")
    plt.show()

# Example on the built-in graph:
plot_graph_with_path(G, path_AF, title="Shortest path A→F highlighted")


### Task 4D (short answer)

Describe your graph (what it represents) and report:
- source, target  
- shortest distance  
- the path

**Your answer here:**  
- 


## 5. When Dijkstra fails: negative weights (10–15 min)

Dijkstra’s “greedy” choice (always expand current smallest distance) is only valid when **weights are non-negative**.

We’ll create a tiny directed graph with a negative edge and see what happens.

In [ ]:
DG = nx.DiGraph()
DG.add_weighted_edges_from([
    ("S","A", 2),
    ("S","B", 5),
    ("A","B",-10),  # negative!
    ("B","T", 2),
    ("A","T", 10),
])

pos = nx.spring_layout(DG, seed=3)
plt.figure(figsize=(6,4))
nx.draw(DG, pos, with_labels=True, node_size=800, arrows=True)
labels = nx.get_edge_attributes(DG, "weight")
nx.draw_networkx_edge_labels(DG, pos, edge_labels=labels)
plt.title("Directed graph with a negative edge")
plt.axis("off")
plt.show()

print("NetworkX shortest path length (Bellman-Ford):",
      nx.shortest_path_length(DG, "S", "T", weight="weight", method="bellman-ford"))
print("Path:", nx.shortest_path(DG, "S", "T", weight="weight", method="bellman-ford"))


In [ ]:
# Try running our Dijkstra (should raise an error)
try:
    dijkstra(DG, "S")
except Exception as e:
    print("As expected, Dijkstra refuses this graph:")
    print(type(e).__name__ + ":", e)


### Task 5A (short answer)

In your own words: why do negative weights break Dijkstra’s logic?  
(Think about the “once visited, distance is final” idea.)

**Your answer here:**  
- 


## 6. Wrap-up reflection (5–8 min)

Write 4–6 sentences:
- What part of Dijkstra was most confusing at first?
- What helped you understand it (trace, picture, code, etc.)?
- One thing you still want to practise.

**Your answer here:**
- 